In [2]:
# Import dependencies
import os
import pickle
# Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode
# for dealing with attachement MIME types
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type
import pandas as pd

# Request all access (permission to read/send/receive emails, manage the inbox, and more)
SCOPES = ['https://mail.google.com/']
our_email = 'viverratesting3@gmail.com'

In [3]:
# Authenicate credentials
def gmail_authenticate():
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

Search for Messages

In [4]:
# Search for messages
def search_messages(service, query):
    result = service.users().messages().list(userId='me',q=query).execute()
    messages = [ ]
    if 'messages' in result:
        messages.extend(result['messages'])
    while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])
    return messages

In [6]:
result = service.users().messages().list(userId='me',q="from: Jordana").execute()

In [7]:
result

{'messages': [{'id': '18ef2172e590ba59', 'threadId': '18ef2172e590ba59'},
  {'id': '18eed1ae4dab59b0', 'threadId': '18eed1ae4dab59b0'},
  {'id': '18ee77bcd57c31eb', 'threadId': '18ee77bcd57c31eb'},
  {'id': '18ee2814addccf73', 'threadId': '18ee2814addccf73'},
  {'id': '18ed2e207da1c432', 'threadId': '18ed2e207da1c432'},
  {'id': '18ece51ac4b15a10', 'threadId': '18ece51ac4b15a10'},
  {'id': '18ec9017d1fc06d0', 'threadId': '18ec9017d1fc06d0'},
  {'id': '18ec374fc2a7514a', 'threadId': '18ec374fc2a7514a'},
  {'id': '18ebe1640cdbf910', 'threadId': '18ebe1640cdbf910'},
  {'id': '18eae99435e7e0fc', 'threadId': '18eae99435e7e0fc'},
  {'id': '18eaa34dc8e31ff3', 'threadId': '18eaa34dc8e31ff3'},
  {'id': '18ea4d3979b00a99', 'threadId': '18ea4d3979b00a99'},
  {'id': '18e8ab7dd1688d99', 'threadId': '18e8ab7dd1688d99'},
  {'id': '18e85d2dfad89c04', 'threadId': '18e85d2dfad89c04'},
  {'id': '18e80eaab784d745', 'threadId': '18e80eaab784d745'},
  {'id': '18e7b8f072ac9c22', 'threadId': '18e7b8f072ac9c22

In [4]:
# Call the search messages function
#search_messages(service, "Joshua")

Reading Messages

In [8]:
# utility functions
def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"


def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

In [9]:
def parse_parts(service, parts, folder_name, message):
    """
    Utility function that parses the content of an email partition
    """
    if parts:
        for part in parts:
            filename = part.get("filename")
            mimeType = part.get("mimeType")
            body = part.get("body")
            data = body.get("data")
            file_size = body.get("size")
            part_headers = part.get("headers")
            if part.get("parts"):
                # recursively call this function when we see that a part
                # has parts inside
                parse_parts(service, part.get("parts"), folder_name, message)
            if mimeType == "text/plain":
                # if the email part is text plain
                if data:
                    text = urlsafe_b64decode(data).decode()
                    print(text)
            elif mimeType == "text/html":
                # if the email part is an HTML content
                # save the HTML file and optionally open it in the browser
                if not filename:
                    filename = "index.html"
                filepath = os.path.join(folder_name, filename)
                print("Saving HTML to", filepath)
                with open(filepath, "wb") as f:
                    f.write(urlsafe_b64decode(data))
            else:
                # attachment other than a plain text or HTML
                for part_header in part_headers:
                    part_header_name = part_header.get("name")
                    part_header_value = part_header.get("value")
                    if part_header_name == "Content-Disposition":
                        if "attachment" in part_header_value:
                            # we get the attachment ID 
                            # and make another request to get the attachment itself
                            print("Saving the file:", filename, "size:", get_size_format(file_size))
                            attachment_id = body.get("attachmentId")
                            attachment = service.users().messages() \
                                        .attachments().get(id=attachment_id, userId='me', messageId=message['id']).execute()
                            data = attachment.get("data")
                            filepath = os.path.join(folder_name, filename)
                            if data:
                                with open(filepath, "wb") as f:
                                    f.write(urlsafe_b64decode(data))

In [10]:
def read_message(service, message):
    """
    This function takes Gmail API `service` and the given `message_id` and does the following:
        - Downloads the content of the email
        - Prints email basic information (To, From, Subject & Date) and plain/text parts
        - Creates a folder for each email based on the subject
        - Downloads text/html content (if available) and saves it under the folder created as index.html
        - Downloads any file that is attached to the email and saves it in the folder created
    """
    msg = service.users().messages().get(userId='me', id=message['id'], format='full').execute()
    # parts can be the message body, or attachments
    payload = msg['payload']
    headers = payload.get("headers")
    parts = payload.get("parts")
    folder_name = "email"
    has_subject = False
    if headers:
        # this section prints email basic info & creates a folder for the email
        for header in headers:
            name = header.get("name")
            value = header.get("value")
            if name.lower() == 'from':
                # we print the From address
                print("From:", value)
            if name.lower() == "to":
                # we print the To address
                print("To:", value)
            if name.lower() == "subject":
                # make our boolean True, the email has "subject"
                has_subject = True
                # make a directory with the name of the subject
                folder_name = clean(value)
                # we will also handle emails with the same subject name
                folder_counter = 0
                while os.path.isdir(folder_name):
                    folder_counter += 1
                    # we have the same folder name, add a number next to it
                    if folder_name[-1].isdigit() and folder_name[-2] == "_":
                        folder_name = f"{folder_name[:-2]}_{folder_counter}"
                    elif folder_name[-2:].isdigit() and folder_name[-3] == "_":
                        folder_name = f"{folder_name[:-3]}_{folder_counter}"
                    else:
                        folder_name = f"{folder_name}_{folder_counter}"
                os.mkdir(folder_name)
                print("Subject:", value)
            if name.lower() == "date":
                # we print the date when the message was sent
                print("Date:", value)
    if not has_subject:
        # if the email does not have a subject, then make a folder with "email" name
        # since folders are created based on subjects
        if not os.path.isdir(folder_name):
            os.mkdir(folder_name)
    parse_parts(service, parts, folder_name, message)
    print("="*50)

In [25]:
# get emails that match the query you specify
results = search_messages(service, "From: Jordana")
print(f"Found {len(results)} results.")
# for each email matched, read it (output plain/text to console & save HTML and attachments)
for msg in results:
    read_message(service, msg)

Found 44 results.
Subject: U.S. reimposes Venezuela oil sanctions
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Thu, 18 Apr 2024 16:42:24 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/us-reimposes-venezuela-oil-sanctions

The U.S. reimposed sanctions on Venezuela’s oil and gas sectors today — specifically, the Treasury Department allowed a temporary waiver to expire — after the Venezuelan Maduro government failed to maintain its commitment  to hold free and fair elections, monitored by international observers.
“Maduro and his representatives did not fully comply with the spirit or the letter of the agreement,” said a senior administration official who spoke with a group of journalists on background to discuss a sensitive diplomatic matter. (New York Times [ https://substack.com/redirect/c398feb8-a3c4-49b3-ab2a-994f5f70427e?j=eyJ1IjoiMzFmdXh4In0.L520X0qdNbvIdiyCX

Subject: Maduro offers to talk with Boric
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Tue, 16 Apr 2024 15:15:38 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/maduro-offers-to-talk-with-boric

Venezuelan President Nicolás Maduro sought to calm growing tensions between his country and Chile — following the February murder of a Venezuelan dissident living in Chile. Yesterday Maduro said he would like to talk directly with his Chilean counterpart, Gabriel Boric, about how to jointly combat international criminal organizations. Boric responded today that he is available. (CNN Chile [ https://substack.com/redirect/3c4b6745-1e43-42aa-a953-a9ad69a84e1a?j=eyJ1IjoiMzFmdXh4In0.L520X0qdNbvIdiyCXDPuD09Laf4PkKgM7ejvjLgFci4 ])
Chilean authorities believe the Venezuelan criminal organization Tren de Aragua carried out the murder of military dissident Ronald Ojeda, and that th

Subject: Haiti transition council established
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Mon, 15 Apr 2024 16:03:42 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/haiti-transition-council-established

Haiti’s government formally ratified a presidential transition council on Friday — but the long-delayed move is only the first step in a fraught path out of the country’s political paralysis. 
“Questions remain over whether the US-backed interim government—dubbed a Presidential Transitional Council—will be able to impose its authority over well-armed gangs that control much of the capital Port-au-Prince,” reports France 24 [ https://substack.com/redirect/fc6517e9-809f-4b54-9181-d1036a774813?j=eyJ1IjoiMzFmdXh4In0.L520X0qdNbvIdiyCXDPuD09Laf4PkKgM7ejvjLgFci4 ].
The new council will need to be sworn in — the government decree stipulates the council be headquartered in

Subject: Mexico's upcoming election
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Fri, 12 Apr 2024 15:16:25 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/mexicos-upcoming-election

Mexico
Mexico’s upcoming general election on June 2 will be a landmark in several ways, reports the New York Times [ https://substack.com/redirect/17aba6ce-c458-4bd0-80b5-2e593c69d4bd?j=eyJ1IjoiMzFmdXh4In0.L520X0qdNbvIdiyCXDPuD09Laf4PkKgM7ejvjLgFci4 ]. “It will be the country’s largest election in terms of voters and seats.” And for the first time Mexicans will elect a female president: the two frontrunners are women.
Security is a top concern — this electoral season might become the country’s most violent ever — and the leading campaigns “are being much more concrete than in past elections” about their security proposals, International Crisis Group’s Falko Ernst told Foreign Policy [

Subject: Mexico wants to expel Ecuador from UN
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Thu, 11 Apr 2024 18:00:23 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/mexico-wants-to-expel-ecuador-from

Mexico is demanding that the United Nations expel Ecuador, as part of its complaint at the International Court of Justice regarding a raid on the Mexican embassy in Quito last week, reports the Associated Press. Mexican President Andrés Manuel López Obrador also said Mexico is demanding a public apology from Ecuador for last week’s raid, reparation of damages and a promise not to do it again.
Ecuador’s controversial embassy raid could be the latest example of the increasing impact of personal political agendas on national foreign policy in Latin America, reports the New York Times [ https://substack.com/redirect/5f31108f-4c7f-4145-9876-e53df4cd063c?j=eyJ1IjoiMzFmdX

...

Subject: U.S. investigated alleged ties between AMLO and cartels, report
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Fri, 23 Feb 2024 19:30:05 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/us-investigated-alleged-ties-between

U.S. law enforcement officers spent years investigating allegations that allies of  Mexican President Andrés Manuel López Obrador met with and took millions of dollars from drug cartels, after he took office in 2018, according to a report published by the New York Times [ https://substack.com/redirect/cb30a29f-f6ab-4200-8ebd-89990a1be036?j=eyJ1IjoiMzFmdXh4In0.L520X0qdNbvIdiyCXDPuD09Laf4PkKgM7ejvjLgFci4 ] yesterday.
The U.S. never opened a formal investigation into AMLO, and the inquiry was shelved after officials “concluded that the U.S. government had little appetite to pursue allegations against the leader of one of America’s top allie

Subject: Latin America Daily Briefing
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Wed, 21 Feb 2024 16:54:33 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/latin-america-daily-briefing-b98

Regional Relations
Brazilian President Luiz Inácio Lula da Silva met with U.S. Secretary of State Antony Blinken in Brasilia yesterday, in the midst of a diplomatic spat between Brazil and Israel. (Reuters [ https://substack.com/redirect/d6841e9d-a007-43c1-8246-4c45be33d6e0?j=eyJ1IjoiMzFmdXh4In0.L520X0qdNbvIdiyCXDPuD09Laf4PkKgM7ejvjLgFci4 ])
The presidents of Colombia and Bolivia, Gustavo Petro and Luis Arce, expressed support for Lula’s comments criticizing Israel’s attacks on Gaza, saying they are a genocide and comparing them to Adolf Hitler’s efforts to exterminate the Jews. (Al Jazeera [ https://substack.com/redirect/aaa08d2b-6df1-4734-9837-d08b74077ab9?j=eyJ1IjoiMzFmdXh

Subject: Moïse's widow charged in relation to his murder
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Tue, 20 Feb 2024 13:43:35 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/moises-widow-charged-in-relation

A Haitian judge charged 51 people in the assassination conspiracy that culminated in the 2021 murder of President Jovenel Moïse — including his widow, Martine, who was wounded in the attack that killed her husband. The indictment obtained by media yesterday accuses the former first lady of complicity in the assassination and associating with criminals. It also charges Claude Joseph, who briefly led the country after Moïse’s death and former national police chief Léon Charles, who led the initial investigations into the murder.
The allegations against Martine Moïse stem from former Justice Ministry official Joseph Felix Badio — who, according to the indictme

Subject: Lula compares Israel's actions to Hitler's
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Mon, 19 Feb 2024 12:08:12 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/lula-compares-israels-actions-to

Brazilian President Luiz Inácio Lula da Silva accused Israel of committing "genocide" against Palestinian civilians in the Gaza Strip, yesterday. He compared Israel’s actions in Gaza to Adolf Hitler’s efforts to exterminate Jews.
Lula said Israel's military campaign was between a "highly prepared army and women and children".
Israeli Prime Minister Benjamin Netanyahu said Lula’s comments “trivialized the Holocaust” and “crossed a red line.” He also accused Lula of being a “virulent anti-Semite.” Israel’s foreign ministry said it summoned Brazil’s ambassador in Israel for a reprimand.
Lula’s words drew praise from Hamas, however, which described the remarks as "a

Subject: Paraguay's Senate expels gov't critic
From: Jordana Timerman from Latin America Daily Briefing <latinamericadailybriefing@substack.com>
To: viverratesting3@gmail.com
Date: Thu, 15 Feb 2024 15:18:16 +0000
View this post on the web at https://latinamericadailybriefing.substack.com/p/paraguays-senate-expels-govt-critic

Paraguay’s Senate expelled Senator Kattya González yesterday, on allegations of administrative misconduct. A member of the center-left National Meeting Party, González is a leading opposition voice in Paraguay’s national politics. (EFE [ https://substack.com/redirect/daf68db1-f20d-4662-868d-75d52395873b?j=eyJ1IjoiMzFmdXh4In0.L520X0qdNbvIdiyCXDPuD09Laf4PkKgM7ejvjLgFci4 ])
She was ousted by members of the Honor faction of the ruling Colorado party, and other lawmakers loyal to former President Horacio Cartes, reports ABC Color [ https://substack.com/redirect/71f4b303-e80b-4e66-adde-046831b2288d?j=eyJ1IjoiMzFmdXh4In0.L520X0qdNbvIdiyCXDPuD09Laf4PkKgM7ejvjLgFci4 ]. (Se

In [13]:
message = service.users().messages().get(userId='me', id='18eed1ae4dab59b0', format='full').execute()

In [23]:
message.get('payload').get('headers')



[{'name': 'Delivered-To', 'value': 'viverratesting3@gmail.com'},
 {'name': 'Received',
  'value': 'by 2002:adf:f2ca:0:b0:343:5370:8f6d with SMTP id d10csp127013wrp;        Wed, 17 Apr 2024 10:29:07 -0700 (PDT)'},
 {'name': 'X-Google-Smtp-Source',
  'value': 'AGHT+IEXMltBn521UfYW494YxWNEs5+YpwaGuaUqge/JGwaQhInyfb9b+anr8fpTCvyZb3UU3dfi'},
 {'name': 'X-Received',
  'value': 'by 2002:ac8:7c53:0:b0:436:83e1:871a with SMTP id o19-20020ac87c53000000b0043683e1871amr168322qtv.5.1713374946732;        Wed, 17 Apr 2024 10:29:06 -0700 (PDT)'},
 {'name': 'ARC-Seal',
  'value': 'i=1; a=rsa-sha256; t=1713374946; cv=none;        d=google.com; s=arc-20160816;        b=GsGAe3U2O+mVri0+tJmzz++Pr7/Y1BqgsGT/pLe0/A9PZVWaig9fcatYl5dp6ERXk5         PPSAZs6OhaPvVfZJyPRrFyC/0I0Ij0tWSHtEG88YSWIo52y3S3e6On6mAsE2it/IF4xZ         1cJHois1QcOP+NGX8+XDUPJ5fvYeji8KJKnOCQLoqFpk2Vd4h4C+osNJiJSFV8kfNxG0         sqd1Qe7S/Hbk/LyXLOSQi7sRKBpjReFywzeyIjiHdJNbgN2qd7mllOfJ9QSigTQLHYR7         gNeHBsZkSn0HPUfTkDKz1KO55+Mg3O77N/Z

In [17]:
type(message)

dict

In [9]:
# Don't need the HTML files
# Have the plain text, get it to Multi-Agent Framework
# Get the mark as read, mark as unread and delete function to work


3/11
Trying some things to filter the plain text


In [12]:
# Attempting to elimiate the email text


Mark emails as read

In [9]:
def mark_as_read(service, query):
    messages_to_mark = search_messages(service, query)
    print(f"Matched emails: {len(messages_to_mark)}")
    return service.users().messages().batchModify(
      userId='me',
      body={
          'ids': [ msg['id'] for msg in messages_to_mark ],
          'removeLabelIds': ['UNREAD']
      }
    ).execute()

Mark emails as unread

In [10]:
def mark_as_unread(service, query):
    messages_to_mark = search_messages(service, query)
    print(f"Matched emails: {len(messages_to_mark)}")
    # add the label UNREAD to each of the search results
    return service.users().messages().batchModify(
        userId='me',
        body={
            'ids': [ msg['id'] for msg in messages_to_mark ],
            'addLabelIds': ['UNREAD']
        }
    ).execute()

Delete emails

In [11]:
def delete_messages(service, query):
    messages_to_delete = search_messages(service, query)
    # it's possible to delete a single message with the delete API, like this:
    # service.users().messages().delete(userId='me', id=msg['id'])
    # but it's also possible to delete all the selected messages with one query, batchDelete
    return service.users().messages().batchDelete(
      userId='me',
      body={
          'ids': [ msg['id'] for msg in messages_to_delete]
      }
    ).execute()